# Segmentation test

In [5]:
import cv2
import numpy as np
import time
from tensorflow.python.keras import Model
from tensorflow.python.keras.models import model_from_json
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.python.keras.utils import to_categorical

In [2]:
json_file = open('hebrew_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("hebrew_model.h5")
print("Loaded model from disk")

Loaded model from disk


In [4]:
FILENAME = r'C:/Users/leohe/Documents/ITC/HIVE/Challenge/sign_language/hsl_alphabet_test.zip'


# LABEL = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9,
#          'K':10, 'L':11, 'M':12, 'N':13, 'O':14, 'P':15, 'Q':16, 'R':17, 'S':18, 'T':19,
#          'U':20, 'V':21, 'W':22, 'X':23, 'Y':24, 'Z':25, 
#          'del':26, 'nothing':27, 'space':28}

LABEL = {'alef': 0, 'bet': 1, 'gimel': 2, 'daled': 3, 
         'hey': 4, 'vav': 5, 'zayin': 6, 'het': 7, 'tet': 8, 
         'yud': 9, 'haf': 10, 'lamed': 11, 'mem': 12, 'nun': 13, 
         'samech': 14, 'ayin': 15, 'pey': 16, 'tzadik': 17, 
         'kuf': 18, 'reish': 19, 'shin': 20, 'taf': 21, 
         'space': 22, 'empty': 23}


SAMPLE_SIZE = 20
N = 200*2*24 // SAMPLE_SIZE
IMG_SIZE = 200 # original image size, to be resized
IMG_CHANNEL = 3 # rgb color scheme

In [ ]:
def idos_filter(img, *options):
    return img

In [ ]:
count = 0
stacked  = np.zeros((N, IMG_SIZE, IMG_SIZE, IMG_CHANNEL), dtype='uint8')
label = np.zeros((N, 1), dtype='uint8')
with ZipFile(FILENAME) as archive:
    for entry in tqdm(archive.infolist()):
        name = str(entry).split("'")[1].replace('/', ' ').split()
        if len(name) == 3:
            with archive.open(entry) as file:
                if count % SAMPLE_SIZE != 0:
                    count += 1
                    continue
                img = idos_filter(np.array(Image.open(file)))
                stacked[count // SAMPLE_SIZE] = img
                label[count // SAMPLE_SIZE] = LABEL[name[1]]
                count += 1

In [ ]:
loss, acc = loaded_model.evaluate(x=stacked, y=to_categorical(label, len(np.unique(label))))
print("Loss {:2f}, Accuracy {:6.2f}%".format(loss, 100 * acc))
targets = np.argmax(to_categorical(label, len(np.unique(label))), axis=-1)
probabilities = loaded_model.predict(x=stacked)
predictions = np.argmax(probabilities, axis=-1)
cm = confusion_matrix(y_true=targets, y_pred=predictions)
print(cm)
classes = np.unique(label)
# plot_confusion_matrix(cm, classes=classes)
report = classification_report(y_true=targets, y_pred=predictions, labels=classes)
print(report)